### Group: Zeru Zhou's Group
### Name: Zeru Zhou
### Email: zeruzhou9@gmail.com
### Country: United States
### University: University of Southern California
### Specialization: Data Science

### Problem Description 
### In this project, I need to build binery classification machine learning models to predict if the bank clients will renew the term deposit or not in order to make corresponding strategies to maintain clients.


### Dataset Information
### The dataset contains 3 parts: Client information, compaign information and social/economical context.
### There are total 41188 instances and 20 columns. 
### The target response is whether the bank client will renew the term deposit.

### Library Importing

In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import RidgeClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.pipeline import Pipeline
#from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, RocCurveDisplay
from sklearn.model_selection import train_test_split
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFECV

## Data uploading

In [2]:
my_df_1 = pd.read_csv('../data/bank/bank-full.csv', sep=';')
my_df_2 = pd.read_csv('../data/bank-additional/bank-additional-full.csv', sep=';')

In [3]:
my_df_2.shape

(41188, 21)

In [4]:
my_df = my_df_2

In [5]:
my_df['y'].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

### As we can see, this is an imbalanced dataset and it has NA values. We need to clean the dataset before moving to any analysis and model building.

## Data Cleaning and imputation

### Drop duplicated rows

In [7]:
my_df.duplicated().value_counts()

False    41176
True        12
dtype: int64

In [8]:
my_df = my_df.drop_duplicates()
my_df.shape

(41176, 21)

### Impute missing values

In [12]:
my_df.isna().any()

age               False
job               False
marital           False
education         False
default           False
housing           False
loan              False
contact           False
month             False
day_of_week       False
duration          False
campaign          False
pdays             False
previous          False
poutcome          False
emp.var.rate      False
cons.price.idx    False
cons.conf.idx     False
euribor3m         False
nr.employed       False
y                 False
dtype: bool

In [13]:
my_df.isnull().any()

age               False
job               False
marital           False
education         False
default           False
housing           False
loan              False
contact           False
month             False
day_of_week       False
duration          False
campaign          False
pdays             False
previous          False
poutcome          False
emp.var.rate      False
cons.price.idx    False
cons.conf.idx     False
euribor3m         False
nr.employed       False
y                 False
dtype: bool

### We can see that there are no NA values if we use the new version of data. I also submitted a version that combine the new data with the old version, then missing values are generated and imputed.

### Now let's concat tables

In [14]:
my_df = pd.concat([my_df_1,my_df_2])

In [15]:
my_df.isnull().any()

age               False
job               False
marital           False
education         False
default           False
balance            True
housing           False
loan              False
contact           False
day                True
month             False
duration          False
campaign          False
pdays             False
previous          False
poutcome          False
y                 False
day_of_week        True
emp.var.rate       True
cons.price.idx     True
cons.conf.idx      True
euribor3m          True
nr.employed        True
dtype: bool

In [16]:
sub = []
for i in my_df.isnull().any().keys():
    if my_df.isnull().any()[i] == False:
        sub.append(i)
sub

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y']

### Drop Duplicates

In [17]:
my_df.duplicated(subset= sub).value_counts()

False    86327
True        72
dtype: int64

In [18]:
my_df = my_df.drop_duplicates()
my_df.shape

(86387, 23)

### Impute missing values

### First, find categorical variables and one-hot encode them

In [19]:
my_df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
balance           float64
housing            object
loan               object
contact            object
day               float64
month              object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
y                  object
day_of_week        object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
dtype: object

In [28]:
my_list = []
for i in my_df.dtypes.keys():
    if my_df.dtypes[i] == 'object':
        my_list.append(i)
my_list.remove('y')
my_list

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome',
 'day_of_week']

In [29]:
df = pd.get_dummies(my_df, columns=my_list)

In [30]:
df.head()

,age,balance,day,duration,campaign,pdays,previous,y,emp.var.rate,cons.price.idx,...,poutcome_failure,poutcome_nonexistent,poutcome_other,poutcome_success,poutcome_unknown,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,58,2143.0,5.0,261,1,-1,0,no,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
1,44,29.0,5.0,151,1,-1,0,no,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
2,33,2.0,5.0,76,1,-1,0,no,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
3,47,1506.0,5.0,92,1,-1,0,no,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
4,33,1.0,5.0,198,1,-1,0,no,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0


In [31]:
x,y = df.drop(columns=['y']), df['y']

### First, we can try simple imputer like filling the value with mean

In [32]:
x = x.drop(columns=['duration'])

In [33]:
x_simple = SimpleImputer(strategy='mean').fit_transform(x)
pd.DataFrame(x_simple).head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,58.0,2143.0,5.0,1.0,-1.0,0.0,0.081922,93.57572,-40.502863,3.621293,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,44.0,29.0,5.0,1.0,-1.0,0.0,0.081922,93.57572,-40.502863,3.621293,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,33.0,2.0,5.0,1.0,-1.0,0.0,0.081922,93.57572,-40.502863,3.621293,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,47.0,1506.0,5.0,1.0,-1.0,0.0,0.081922,93.57572,-40.502863,3.621293,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,33.0,1.0,5.0,1.0,-1.0,0.0,0.081922,93.57572,-40.502863,3.621293,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Since this is high-dimensional dataset with almost 100k instances, using simple imputer may lead to huge error. So we try iterative imputer using round robin algorithm

In [34]:
x_iter = IterativeImputer(n_nearest_features=50).fit_transform(x)
x_iter_df = pd.DataFrame(x_iter)

In [35]:
x_iter_df.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,58.0,2143.0,5.0,1.0,-1.0,0.0,-0.035837,93.442005,-37.888759,3.754859,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,44.0,29.0,5.0,1.0,-1.0,0.0,-0.018642,93.458280,-38.280911,3.726448,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,33.0,2.0,5.0,1.0,-1.0,0.0,-0.068157,93.433353,-38.706980,3.674341,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,47.0,1506.0,5.0,1.0,-1.0,0.0,0.009976,93.514050,-38.110792,3.711050,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,33.0,1.0,5.0,1.0,-1.0,0.0,-0.017609,93.505272,-37.700385,3.742608,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### After filling all the NAs, the next step is to eliminate some outliers

In [36]:
x_iter_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,...,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000,86387.000000
mean,40.501314,1056.742722,14.127262,2.670437,479.792504,0.386181,0.026930,93.545136,-39.493067,3.647854,...,0.105953,0.411532,0.021300,0.033385,0.427831,0.090592,0.098533,0.099760,0.093602,0.094158
std,10.534612,2272.444678,6.717591,2.947981,483.824356,1.713173,1.114288,0.461661,4.072957,1.234497,...,0.307780,0.492114,0.144382,0.179640,0.494767,0.287030,0.298036,0.299682,0.291276,0.292050
min,17.000000,-8019.000000,1.000000,1.000000,-1.000000,0.000000,-3.400000,92.201000,-50.800000,-17.133469,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.000000,145.562937,10.353947,1.000000,-1.000000,0.000000,-0.274125,93.316129,-42.000000,3.225901,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,39.000000,571.000000,13.573327,2.000000,246.000000,0.000000,-0.039346,93.466319,-40.035852,3.857398,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,48.000000,1186.220266,17.038271,3.000000,999.000000,0.000000,1.100000,93.918000,-36.400000,4.857000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,98.000000,102127.000000,36.159108,63.000000,999.000000,275.000000,11.260477,129.876347,-11.286659,5.045000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
x_iter_df['y'] = y.values

In [38]:
for i in x_iter_df.columns:
    if i == 'y':
        continue
    q_low = x_iter_df[i].quantile(0.01)
    q_high = x_iter_df[i].quantile(0.99)
    x_iter_df = x_iter_df.loc[(x_iter_df[i] <= q_high) & (x_iter_df[i] >= q_low)]

In [39]:
x_iter_df.shape

(74632, 73)

### Here above, For each feature, I only dropped very extreme values that not belong to the central 99%. It is also feasible to use q1, q3, and 1.5 IQR to detect outliers but I don't want that much data lose.

# I've already started model building. Since purpose for this week is data cleaning, I'll show the rest of the code in the following weeks. Cheers!